In [1]:
from agential.cog.lats.strategies.qa import LATSQAStrategy 
from agential.llm.llm import LLM
from agential.cog.fewshots.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,
)

llm = LLM("gpt-3.5-turbo")

question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

agent = LATSQAStrategy(
    llm=llm, 
    n_samples=2, 
    max_reflections=4, 
    depth_limit=5,
    max_unique=5,
    cache_values=True,
)

out = agent.generate(
    question=question,
    key=key,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    value_examples=HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE,
    prompt=LATS_INSTRUCTION_HOTPOTQA,
    reflect_prompt=LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    value_prompt=LATS_VALUE_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reflect_additional_keys={},
    value_additional_keys={},
    max_iterations=1,
    reset=True
)

In [5]:
from agential.cog.lats.functional import get_node_trajectory_qa


print(get_node_trajectory_qa(out.answer))


Thought 1: I need to search for VIVA Media AG's new acronym after changing their name in 2004.
Action 1: Search[VIVA Media AG new acronym]
Observation 1: Could not find [VIVA Media AG new acronym]. Similar: ['VIVA (German TV channel)', 'Holden Torana', 'Earth Liberation Front', 'Andrés Manuel López Obrador', 'List of Nintendo DS games (Q–Z)', 'HSBC']
Thought 2: I should try searching for the new name of VIVA Media AG instead.
Action 2: Search[VIVA Media AG new name]
Observation 2: Could not find [VIVA Media AG new name]. Similar: ['Vauxhall Viva', 'MTV Music (Polish TV channel)', 'Sartorius AG', 'Paramount International Networks', 'Flying Bark Productions', 'Lindt', 'Luann de Lesseps', 'Sigg', 'Puma (brand)', 'Opel Astra']
Thought 3: I need to refine my search keywords to find the new acronym or new name of VIVA Media AG after 2004.
Action 3: Search[VIVA Media AG name change 2004]
Observation 3: Could not find [VIVA Media AG name change 2004]. Similar: ['Vauxhall Viva', 'GfK Entertain